# Short Term Rentals - Exploratory Data Analysis

Now we're going to see what we've imported. As with the previous notebook let's import py2neo and pandas:

In [ ]:
%matplotlib notebook

from py2neo import Graph
import pandas as pd

import matplotlib 
import matplotlib.pyplot as plt
plt.style.use('fivethirtyeight')

In [ ]:
graph = Graph("bolt://localhost", auth=("neo4j", "neo"))

Now we can run the following query to check how many nodes our database contains:

In [ ]:
query = """
CALL db.schema() 
"""

graph.run(query).data()

This query returns all the labels in the database, any constraints they have, as well as relationship types.

We could also run this query in the Neo4j Browser if we want to see a visual representation:

<img align="left" src="images/airbnb-graph.svg" width="500px" />

In [ ]:
query = """
MATCH () 
RETURN COUNT(*) AS nodeCount
"""

graph.run(query).to_data_frame()

Let's drill down a bit. What types of nodes do we have?

In [ ]:
result = {"label": [], "count": []}
for label in graph.run("CALL db.labels()").to_series():
    query = f"MATCH (:`{label}`) RETURN count(*) as count"
    count = graph.run(query).to_data_frame().iloc[0]['count']
    result["label"].append(label)
    result["count"].append(count)
nodes_df = pd.DataFrame(data=result)
nodes_df.sort_values("count")

We can visualize this counts using matplotlib with the following code:

In [ ]:
nodes_df.plot(kind='bar', x='label', y='count', legend=None, title="Node Cardinalities")
plt.yscale("log")
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

And what types of relationships?

In [ ]:
result = {"relType": [], "count": []}
for relationship_type in graph.run("CALL db.relationshipTypes()").to_series():
    query = f"MATCH ()-[:`{relationship_type}`]->() RETURN count(*) as count"
    count = graph.run(query).to_data_frame().iloc[0]['count']
    result["relType"].append(relationship_type)
    result["count"].append(count)
rels_df = pd.DataFrame(data=result)
rels_df.sort_values("count")

We can visualize this counts using matplotlib with the following code:

In [ ]:
rels_df.plot(kind='bar', x='relType', y='count', legend=None, title="Relationship Cardinalities")
plt.yscale("log")
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

Now let's explore the neighborhood data:

In [ ]:
exploratory_query = """
MATCH (n:Neighborhood)<-[:IN_NEIGHBORHOOD]-(l:Listing)-[:HAS]->(a:Amenity) 
RETURN n.name AS neighborhood, l.name AS name, collect(a.name) AS amenities, l.price AS price 
LIMIT 25
"""

graph.run(exploratory_query).to_data_frame()

What does the distribution of prices in the dataset look like? We can use the `describe` function to work this out:

In [ ]:
query = """
MATCH (l:Listing)
RETURN l.price AS price
"""

price_df = graph.run(query).to_data_frame()
price_df.describe()

This returns some descriptive statistics that allow us to get an understanding of how prices are distributed. We can see that the average price is 139 per night, but the maximum price is 999 - there's clearly a lot of variation in prices!

In [ ]:
plt.style.use('fivethirtyeight')
plt.hist(price_df['price'].dropna())
plt.tight_layout()
plt.show() 

We have a very long tail going on here - the majority of listings are priced at under 200, but then there are a few properties that cost much more than this.

What are the most expensive places to live?

In [ ]:
query = """
MATCH (l:Listing)-[:IN_NEIGHBORHOOD]->(n:Neighborhood)
WITH n, avg(l.price) AS averagePrice
RETURN n.id AS zip, n.name AS neighborhood, averagePrice
"""

price_df = graph.run(query).to_data_frame().sort_values("averagePrice", ascending=False)
price_df.head(10)

The variation in average price by neighborhood is easier to see in a chart:

In [ ]:
price_df.head(30).plot(kind='bar', x='zip', y='averagePrice', legend=None, title="Average price")
plt.tight_layout()
plt.show()

## Exercise 

* Can you create a similar chart showing the areas which are offering the largest number of bedrooms?
* What about bathrooms?
* What about the number of listings per neighborhood?